In [ ]:
import numpy as np
import random
import gurobipy as gp

In [ ]:
def generate_instance(n, p, mu, sigma):
    # Check if input mu is a 1D array of means for each point
    if len(mu) != n:
        raise ValueError("mean must be a 1D array with n elements")
    points = np.random.multivariate_normal(mean=mu, cov=sigma, size=n)
    demand = np.abs(points[:, 0])
    capacity = np.abs(points[:, 1])
    distances = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            distances[i, j] = np.linalg.norm(points[i, :] - points[j, :])
    return points, demand, capacity, distances

In [ ]:
points, demand, capacity, distances = generate_instance(n=10, p=2, mu = [5 for i in range(10)], sigma=np.eye(10))

In [ ]:
def solve_ccp(points, demand, capacity, distances, λ, M):
    # points: points generated by the instance
    # demand: demand for each point
    # capacity: capacity for each cluster
    # distances: distances between each point
    # λ: weight of the objective function
    # M: a big positive number

    n = len(points) # number of points
    p = len(capacity) # maximum number of clusters

    # Create a new model
    model = gp.Model("Capacitated Clustering Problem")

    # Add decision variables
    x = {}
    y = {}
    for i in range(n):
        for j in range(p):
            x[i, j] = model.addVar(vtype=gp.GRB.BINARY, name="x_{}_{}".format(i, j))
        y[i] = model.addVar(vtype=gp.GRB.BINARY, name="y_{}".format(i))

    # Set objective function
    obj = gp.quicksum(λ * demand[i] * y[i] + (1 - λ) * distances[i, j] * x[i, j] for i in range(n) for j in range(p))
    model.setObjective(obj, sense=gp.GRB.MINIMIZE)

    # Add constraints
    for i in range(n):
        model.addConstr(gp.quicksum(x[i, j] for j in range(p)) == 1, name="c1_{}".format(i))
        for j in range(p):
            model.addConstr(x[i, j] <= y[j], name="c2_{}_{}".format(i, j))
    for j in range(p):
        model.addConstr(gp.quicksum(demand[i] * x[i, j] for i in range(n)) <= capacity[j], name="c3_{}".format(j))
    model.addConstr(gp.quicksum(y[i] for i in range(n)) <= p, name="c4")

    # Optimize the model
    model.optimize()

    # Return the optimal objective value
    return model.ObjVal

In [ ]:
def generate_weights(n, mu, sigma):
    """
    Generate n random realizations of weights from a normal distribution
    with mean mu and standard deviation sigma.

    Parameters
    ----------
    n : int
        Number of realizations
    mu : float
        Mean of the normal distribution
    sigma : float
        Standard deviation of the normal distribution

    Returns
    -------
    weights : numpy.ndarray
        A numpy array of size (n, 1) containing the random realizations of weights.
    """
    weights = np.random.normal(mu, sigma, (n, 1))
    return weights

In [ ]:
# Define the number of points and the maximum number of clusters
n = 100
p = 10

# Generate random realizations of weights
mu = 100
sigma = 10
weights = generate_weights(n, mu, sigma)

# Define the distances between points
distances = np.random.rand(n, n)

# Define the capacities of the clusters
capacities = np.random.randint(1, 10, (p, 1))

# Create the Gurobi model
m = gp.Model("Capacitated Clustering Problem")

# Define the decision variables
x = {}
y = {}
for i in range(n):
    for j in range(p):
        x[i, j] = m.addVar(vtype=gp.GRB.BINARY, name="x_{}_{}".format(i, j))
    y[i] = m.addVar(vtype=gp.GRB.BINARY, name="y_{}".format(i))

# Update the model to include the decision variables
m.update()

# Define the objective function
obj = gp.quicksum(distances[i, j] * x[i, j] for i in range(n) for j in range(p)) + \
      gp.quicksum(weights[i] * (1 - y[i]) for i in range(n))
m.setObjective(obj, gp.GRB.MINIMIZE)

# Add constraints
for i in range(n):
    m.addConstr(gp.quicksum(x[i, j] for j in range(p)) == 1)

for j in range(p):
    m.addConstr(gp.quicksum(y[i] * x[i, j] for i in range(n)) <= capacities[j])

for i in range(n):
    for j in range(p):
        m.addConstr(x[i, j] <= y[j])

# Optimize the model
m.optimize()

# Print the solution
for v in m.getVars():
    print("{} = {}".format(v.varName, v.x))